In [1]:
!pip install python-dotenv
!pip install pandas

  Using cached pandas-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.1 MB)
  Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
  Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)


In [2]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [49]:
from appwrite.client import Client
from appwrite.services.databases import Databases
from appwrite.id import ID
from dotenv import load_dotenv
import os

load_dotenv()

In [31]:
client = Client()
client.set_endpoint(os.getenv('APPWRITE_API_ENDPOINT'))
client.set_project(os.getenv('APPWRITE_PROJECT'))
client.set_key(s.getenv('APPWRITE_API_KEY'))

In [32]:
fireSaleDb = databases.create(
    database_id=ID.unique(),
    name='fire-sale-db'
)

#### User Profile Collectin

In [40]:
userProfileCollection = None

def prepare_user_profile_collection():
  global userProfileCollection

  userProfileCollection = databases.create_collection(
    database_id=fireSaleDb['$id'],
    collection_id=ID.unique(),
    name='user-profile'
  )

  databases.create_string_attribute(
    database_id=fireSaleDb['$id'],
    collection_id=userProfileCollection['$id'],
    key='user_id',
    size=255,
    required=True
  )

  databases.create_string_attribute(
    database_id=fireSaleDb['$id'],
    collection_id=userProfileCollection['$id'],
    key='name',
    size=255,
    required=False
  )

  databases.create_string_attribute(
    database_id=fireSaleDb['$id'],
    collection_id=userProfileCollection['$id'],
    key='email',
    size=255,
    required=False
  )
    
  databases.create_string_attribute(
    database_id=fireSaleDb['$id'],
    collection_id=userProfileCollection['$id'],
    key='label',
    size=255,
    required=True
  )
    
  databases.create_integer_attribute(
    database_id=fireSaleDb['$id'],
    collection_id=userProfileCollection['$id'],
    key='age',
    required=False,
    min=18,
    max=150
  )
    
  databases.create_datetime_attribute(
    database_id=fireSaleDb['$id'],
    collection_id=userProfileCollection['$id'],
    key='signup_date',
    required=True
  )

#### Social Media Collection

In [41]:
socialMediaCollection = None

def prepare_social_media_collection():
  global socialMediaCollection

  socialMediaCollection = databases.create_collection(
    database_id=fireSaleDb['$id'],
    collection_id=ID.unique(),
    name='social-media'
  )

  databases.create_string_attribute(
    database_id=fireSaleDb['$id'],
    collection_id=socialMediaCollection['$id'],
    key='guid',
    size=255,
    required=True
  )
    
  databases.create_string_attribute(
    database_id=fireSaleDb['$id'],
    collection_id=socialMediaCollection['$id'],
    key='user_id',
    size=255,
    required=False
  )

  databases.create_string_attribute(
    database_id=fireSaleDb['$id'],
    collection_id=socialMediaCollection['$id'],
    key='platform',
    size=255,
    required=False
  )

  databases.create_integer_attribute(
    database_id=fireSaleDb['$id'],
    collection_id=socialMediaCollection['$id'],
    key='post_count',
    required=False,
    min=0
  )  


  databases.create_integer_attribute(
    database_id=fireSaleDb['$id'],
    collection_id=socialMediaCollection['$id'],
    key='like_count',
    required=False,
    min=0
  )  

  databases.create_string_attribute(
    database_id=fireSaleDb['$id'],
    collection_id=socialMediaCollection['$id'],
    key='group_memberships',
    required=False, 
    size=131072
  )    
    
  databases.create_integer_attribute(
    database_id=fireSaleDb['$id'],
    collection_id=socialMediaCollection['$id'],
    key='follower_count',
    required=False,
    min=0
  ) 
    
  databases.create_string_attribute(
    database_id=fireSaleDb['$id'],
    collection_id=socialMediaCollection['$id'],
    key='top_interests',
    required=False, 
    size=131072
  )  
    
  databases.create_datetime_attribute(
    database_id=fireSaleDb['$id'],
    collection_id=socialMediaCollection['$id'],
    key='last_active',
    required=True
  )

#### Communication Collection

In [42]:
communicationCollection = None

def prepare_communication_collection():
  global communicationCollection

  communicationCollection = databases.create_collection(
    database_id=fireSaleDb['$id'],
    collection_id=ID.unique(),
    name='communication'
  )

  databases.create_string_attribute(
    database_id=fireSaleDb['$id'],
    collection_id=communicationCollection['$id'],
    key='guid',
    size=255,
    required=True
  )
    
  databases.create_string_attribute(
    database_id=fireSaleDb['$id'],
    collection_id=communicationCollection['$id'],
    key='user_id',
    size=255,
    required=False
  )

  databases.create_string_attribute(
    database_id=fireSaleDb['$id'],
    collection_id=communicationCollection['$id'],
    key='medium',
    size=255,
    required=False
  )
    
  databases.create_string_attribute(
    database_id=fireSaleDb['$id'],
    collection_id=communicationCollection['$id'],
    key='contacted_user_id',
    size=255,
    required=False
  )

  databases.create_integer_attribute(
    database_id=fireSaleDb['$id'],
    collection_id=communicationCollection['$id'],
    key='interaction_count',
    required=False,
    min=0
  )  

  databases.create_string_attribute(
    database_id=fireSaleDb['$id'],
    collection_id=communicationCollection['$id'],
    key='topics',
    required=False, 
    size=131072
  )    
    
  databases.create_datetime_attribute(
    database_id=fireSaleDb['$id'],
    collection_id=communicationCollection['$id'],
    key='last_contact_date',
    required=False
  )

#### Location Collection

In [43]:
locationCollection = None

def prepare_location_collection():
  global locationCollection

  locationCollection = databases.create_collection(
    database_id=fireSaleDb['$id'],
    collection_id=ID.unique(),
    name='location'
  )

  databases.create_string_attribute(
    database_id=fireSaleDb['$id'],
    collection_id=locationCollection['$id'],
    key='guid',
    size=255,
    required=True
  )
    
  databases.create_string_attribute(
    database_id=fireSaleDb['$id'],
    collection_id=locationCollection['$id'],
    key='user_id',
    size=255,
    required=False
  )

  databases.create_string_attribute(
    database_id=fireSaleDb['$id'],
    collection_id=locationCollection['$id'],
    key='location_name',
    size=255,
    required=False
  )
    
  databases.create_float_attribute(
    database_id=fireSaleDb['$id'],
    collection_id=locationCollection['$id'],
    key='latitude',
    required=False
  )
    
  databases.create_float_attribute(
    database_id=fireSaleDb['$id'],
    collection_id=locationCollection['$id'],
    key='longitude',
    required=False
  )
   
  databases.create_datetime_attribute(
    database_id=fireSaleDb['$id'],
    collection_id=locationCollection['$id'],
    key='check_in_time',
    required=False
  )
    
  databases.create_string_attribute(
    database_id=fireSaleDb['$id'],
    collection_id=locationCollection['$id'],
    key='companions',
    required=False, 
    size=131072
  )    
    

#### Behavioral Metadata

In [44]:
behaviorMetadataCollection = None

def prepare_behaviour_metadata_collection():
  global behaviorMetadataCollection

  behaviorMetadataCollection = databases.create_collection(
    database_id=fireSaleDb['$id'],
    collection_id=ID.unique(),
    name='behavioral-metadata'
  )

  databases.create_string_attribute(
    database_id=fireSaleDb['$id'],
    collection_id=behaviorMetadataCollection['$id'],
    key='guid',
    size=255,
    required=True
  )
    
  databases.create_string_attribute(
    database_id=fireSaleDb['$id'],
    collection_id=behaviorMetadataCollection['$id'],
    key='user_id',
    size=255,
    required=False
  )

  databases.create_string_attribute(
    database_id=fireSaleDb['$id'],
    collection_id=behaviorMetadataCollection['$id'],
    key='device_type',
    size=255,
    required=False
  )

  databases.create_string_attribute(
    database_id=fireSaleDb['$id'],
    collection_id=behaviorMetadataCollection['$id'],
    key='active_hours',
    size=255,
    required=False
  )
      
  databases.create_float_attribute(
    database_id=fireSaleDb['$id'],
    collection_id=behaviorMetadataCollection['$id'],
    key='average_daily_screen_time',
    required=False
  )

  databases.create_string_attribute(
    database_id=fireSaleDb['$id'],
    collection_id=behaviorMetadataCollection['$id'],
    key='preferred_app_categories',
    required=False, 
    size=131072
  )  

#### InferredProfile

In [45]:
inferredProfileCollection = None

def prepare_inferred_profile_collection():
  global inferredProfileCollection

  inferredProfileCollection = databases.create_collection(
    database_id=fireSaleDb['$id'],
    collection_id=ID.unique(),
    name='inferred-profile'
  )
    
  databases.create_string_attribute(
    database_id=fireSaleDb['$id'],
    collection_id=inferredProfileCollection['$id'],
    key='user_id',
    size=255,
    required=False
  )

  databases.create_float_attribute(
    database_id=fireSaleDb['$id'],
    collection_id=inferredProfileCollection['$id'],
    key='influence_score',
    required=False
  )

    
  databases.create_float_attribute(
    database_id=fireSaleDb['$id'],
    collection_id=inferredProfileCollection['$id'],
    key='trust_score',
    required=False
  )


  databases.create_string_attribute(
    database_id=fireSaleDb['$id'],
    collection_id=inferredProfileCollection['$id'],
    key='hidden_roles',
    required=False, 
    size=131072
  )  

  databases.create_string_attribute(
    database_id=fireSaleDb['$id'],
    collection_id=inferredProfileCollection['$id'],
    key='hidden_communities',
    required=False, 
    size=131072
  )  

#### Create the database collections

In [46]:
if __name__ == "__main__":
  prepare_user_profile_collection()
  prepare_social_media_collection()
  prepare_communication_collection()
  prepare_location_collection()
  prepare_behaviour_metadata_collection()
  prepare_inferred_profile_collection()

In [4]:
def seed_database():
  testTodo1 = {
    'title': "Buy apples",
    'description': "At least 2KGs",
    'isComplete': True
  }

  testTodo2 = {
    'title': "Wash the apples", 
    'isComplete': True
  }

  testTodo3 = {
    'title': "Cut the apples",
    'description': "Don\'t forget to pack them in a box",
    'isComplete': False
  }

  databases.create_document(
    database_id=fireSaleDb['$id'],
    collection_id=userProfileCollection['$id'],
    document_id=ID.unique(),
    data=testTodo1
  )

  databases.create_document(
    database_id=fireSaleDb['$id'],
    collection_id=userProfileCollection['$id'],
    document_id=ID.unique(),
    data=testTodo2
  )

  databases.create_document(
    database_id=fireSaleDb['$id'],
    collection_id=userProfileCollection['$id'],
    document_id=ID.unique(),
    data=testTodo3
  )


In [5]:
def get_todos():
  todos = databases.list_documents(
    database_id=fireSaleDb['$id'],
    collection_id=userProfileCollection['$id']
  )
  for todo in todos['documents']:
    print(f"Title: {todo['title']}\nDescription: {todo['description']}\nIs Todo Complete: {todo['isComplete']}\n\n")

if __name__ == "__main__":
  prepare_database()
  seed_database()
  get_todos()


Title: Buy apples
Description: At least 2KGs
Is Todo Complete: True


Title: Wash the apples
Description: This is a test description.
Is Todo Complete: True


Title: Cut the apples
Description: Don't forget to pack them in a box
Is Todo Complete: False




In [29]:
result = databases.delete(
    database_id = '680fc5380017320a8568'
)

# Data Processing

In [4]:
import pandas as pd

# Replace 'your_file.csv' with your actual file path
file_path = 'dataset/fbpac-ads-en-US.csv'

# Read the CSV file
df = pd.read_csv(file_path)
headers = df.columns.tolist()

# Display the first 5 rows with headers
df.head()
print(headers)

['id', 'html', 'political', 'not_political', 'title', 'message', 'thumbnail', 'created_at', 'updated_at', 'lang', 'images', 'impressions', 'political_probability', 'targeting', 'suppressed', 'targets', 'advertiser', 'entities', 'page', 'lower_page', 'targetings', 'paid_for_by', 'targetedness', 'listbuilding_fundraising_proba']


#### Create a smaller file

In [6]:
import pandas as pd

# Read original CSV
file_path = 'dataset/fbpac-ads-en-US.csv'

df = pd.read_csv(file_path)

# Select columns you want to keep (adjust names as needed)
selected_columns = ['title', 'message', 'advertiser', 'entities', 'page']  

# Create new DataFrame with only selected columns
new_df = df[selected_columns]

# Save to new CSV
new_df.to_csv('dataset/compressed_ad_data.csv', index=False)

print(f"New CSV created with columns: {selected_columns}")

New CSV created with columns: ['title', 'message', 'advertiser', 'entities', 'page']
